In [6]:
!pip install langchain docarray tiktoken

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [1]:
import dotenv

dotenv.load_dotenv()

True

In [2]:
knowlege_base=["harrison worked at kensho", "Harrison likes cookies", "Harrisons favourite color is magenta"]

In [3]:
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain_community.vectorstores import Chroma

model = "all-MiniLM-L6-v2"
embedding_function = SentenceTransformerEmbeddings(model_name=model)

vectorstore = Chroma.from_texts(texts=knowlege_base, embedding=embedding_function)
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

In [4]:
retriever.invoke("favourite color")

[Document(page_content='Harrisons favourite color is magenta')]

In [19]:
# Requires:
# 

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import AzureChatOpenAI
from langchain_openai.embeddings import AzureOpenAIEmbeddings


template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
chat = AzureChatOpenAI(
    openai_api_version="2024-02-15-preview",
    azure_deployment="gpt-35-turbo",
    temperature=0.2)
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | chat | output_parser

'Harrison worked at Kensho.'

In [22]:
chain.invoke("where did harrison work?")

'Harrison worked at Kensho.'

In [20]:
chain.invoke("What does Harrison eat?")

'Harrison eats cookies.'

In [21]:
chain.invoke("In which color would Harrison paint is house?")

'Harrison would paint his house magenta.'